In [1]:
import sys
sys.path.append('../')

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import SubsetRandomSampler
import numpy as np
import random

import numpy as np
import pandas as pd
import os
import glob
import tqdm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import sklearn

eps = np.finfo(float).eps

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [30]:
batch_size = 4
feat_dim = 512
tau = 1
x_i = torch.randn(batch_size, feat_dim)
x_j = x_i + 0.05 * torch.randn(batch_size, feat_dim)
x = torch.cat((x_i, x_j), dim=0)

In [31]:
sim_mat_nom = torch.mm(x, x.T)
sim_mat_denom = torch.mm(torch.norm(x, dim=1).unsqueeze(1), torch.norm(x, dim=1).unsqueeze(1).T)

sim_mat = sim_mat_nom / sim_mat_denom.clamp(min=1e-16)
sim_mat = torch.exp(sim_mat / tau)

print(sim_mat.size())

torch.Size([8, 8])


In [32]:
print(sim_mat)

tensor([[2.7183, 0.9720, 1.0545, 1.1027, 2.7150, 0.9737, 1.0540, 1.1049],
        [0.9720, 2.7183, 0.9984, 0.9923, 0.9764, 2.7149, 0.9982, 0.9930],
        [1.0545, 0.9984, 2.7183, 0.9384, 1.0507, 0.9996, 2.7149, 0.9388],
        [1.1027, 0.9923, 0.9384, 2.7183, 1.1034, 0.9916, 0.9388, 2.7148],
        [2.7150, 0.9764, 1.0507, 1.1034, 2.7183, 0.9780, 1.0505, 1.1057],
        [0.9737, 2.7149, 0.9996, 0.9916, 0.9780, 2.7183, 0.9995, 0.9923],
        [1.0540, 0.9982, 2.7149, 0.9388, 1.0505, 0.9995, 2.7183, 0.9392],
        [1.1049, 0.9930, 0.9388, 2.7148, 1.1057, 0.9923, 0.9392, 2.7183]])


In [53]:
i_ind = torch.arange(0, batch_size)
j_ind = torch.arange(batch_size, batch_size * 2)

diag_ind = torch.eye(batch_size * 2).bool()
sim_mat = sim_mat.masked_fill_(diag_ind, 0)

left_score = []
right_score = []
losses = []
print(sim_mat)
for i, j in zip(i_ind, j_ind):
    print("{}-{}".format(i, j))
    left = sim_mat[i][j] / torch.sum(sim_mat[i, :])
    right = sim_mat[j][i] / torch.sum(sim_mat[j, :])
    left_score.append(left)
    right_score.append(right)
    final_score = -torch.log(right + right)
    losses.append(final_score)

torch.mean(torch.tensor(losses))

tensor([[0.0000, 0.9720, 1.0545, 1.1027, 2.7150, 0.9737, 1.0540, 1.1049],
        [0.9720, 0.0000, 0.9984, 0.9923, 0.9764, 2.7149, 0.9982, 0.9930],
        [1.0545, 0.9984, 0.0000, 0.9384, 1.0507, 0.9996, 2.7149, 0.9388],
        [1.1027, 0.9923, 0.9384, 0.0000, 1.1034, 0.9916, 0.9388, 2.7148],
        [2.7150, 0.9764, 1.0507, 1.1034, 0.0000, 0.9780, 1.0505, 1.1057],
        [0.9737, 2.7149, 0.9996, 0.9916, 0.9780, 0.0000, 0.9995, 0.9923],
        [1.0540, 0.9982, 2.7149, 0.9388, 1.0505, 0.9995, 0.0000, 0.9392],
        [1.1049, 0.9930, 0.9388, 2.7148, 1.1057, 0.9923, 0.9392, 0.0000]])
0-4
1-5
2-6
3-7


tensor(0.4803)

In [46]:
left_score

[tensor(0.3024), tensor(0.3140), tensor(0.3122), tensor(0.3091)]

In [47]:
right_score

[tensor(0.3023), tensor(0.3139), tensor(0.3122), tensor(0.3089)]

In [48]:
losses

[tensor(0.6048), tensor(0.6279), tensor(0.6245), tensor(0.6180)]

In [50]:
2.7150 / torch.sum(sim_mat[0, :])

tensor(0.3024)